In [1]:
from scipy.special import dtype
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from pathlib import Path

In [37]:
current_file = Path().resolve()

src_dir = current_file
while src_dir.name != "src" and src_dir != src_dir.parent:
    src_dir = src_dir.parent

In [9]:
data_dir = Path(src_dir / 'data' / 'classification_data')

In [10]:
# import shutil
# from sklearn.model_selection import train_test_split
# 
# output_dir = Path(src_dir / 'data' / 'resnet_data')
# train_dir = output_dir / "train"
# valid_dir = output_dir / "valid"
# 
# for split in [train_dir, valid_dir]:
#     split.mkdir(parents=True, exist_ok=True)
#     
# 
# for class_dir in data_dir.iterdir():
#     if class_dir.is_dir():
#         images = list(class_dir.glob("*.jpg")) + list(class_dir.glob("*.png"))
#         train_images, valid_images = train_test_split(images, test_size=0.2, random_state=42)
#         
#         for img_path in train_images:
#             dest_dir = train_dir / class_dir.name
#             dest_dir.mkdir(parents=True, exist_ok=True)
#             shutil.copy(img_path, dest_dir)
#         
#         for img_path in valid_images:
#             dest_dir = valid_dir / class_dir.name
#             dest_dir.mkdir(parents=True, exist_ok=True)
#             shutil.copy(img_path, dest_dir)


In [62]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),      
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [63]:
# resnet_data_dir = Path(src_dir / 'data' / 'resnet_data')
resnet_data_dir = Path(src_dir / 'data' / 'fall_detection_v1')


In [64]:
image_datasets = {
    'train': datasets.ImageFolder(root=f"{resnet_data_dir}/train", transform=data_transforms['train']),
    'val': datasets.ImageFolder(root=f"{resnet_data_dir}/valid", transform=data_transforms['val'])
}

dataloaders = {
    'train': DataLoader(image_datasets['train'], batch_size=32, shuffle=True, num_workers=4),
    'val': DataLoader(image_datasets['val'], batch_size=32, shuffle=False, num_workers=4)
}

In [67]:
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

print(dataset_sizes)
class_names

{'train': 3192, 'val': 1140}


['Fall Detected', 'Not Fall']

In [68]:
import torch
import torch.nn as nn
from torchvision import models

model = models.resnet50(pretrained=True)

num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)

device = torch.device("mps")
model = model.to(device)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [73]:
import torch.optim as optim
from sklearn.utils import class_weight
import numpy as np

labels = [sample[1] for sample in image_datasets['train'].samples]
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(labels), y=labels)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32, device=device)


criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)
optimizer = optim.Adam(model.parameters(), lr=0.001)

scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)


In [77]:
import copy

def train_model(model, dataloaders, criterion, optimizer, scheduler, num_epochs=25):
    best_model_weights = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
            
            running_loss = 0.0
            running_corrects = 0
            
            for inputs, labels in dataloaders[phase]:
                inputs, labels = inputs.to(device), labels.to(device)
                
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        
                running_loss += float(loss.item() * inputs.size(0)) 
                running_corrects += torch.sum(preds == labels.data).float()  

                
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects / dataset_sizes[phase]



            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
                    
            
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_weights = copy.deepcopy(model.state_dict())

    print(f'Best val Acc: {best_acc:.4f}')
    model.load_state_dict(best_model_weights)
    return model


In [78]:
model = train_model(model, dataloaders, criterion, optimizer, scheduler, num_epochs=25)


train Loss: 0.1329 Acc: 0.9549
val Loss: 0.1310 Acc: 0.9465
train Loss: 0.1412 Acc: 0.9445
val Loss: 0.1561 Acc: 0.9430
train Loss: 0.0767 Acc: 0.9715
val Loss: 0.2102 Acc: 0.9474
train Loss: 0.1064 Acc: 0.9615
val Loss: 0.1185 Acc: 0.9605
train Loss: 0.0704 Acc: 0.9746
val Loss: 0.1510 Acc: 0.9474
train Loss: 0.0429 Acc: 0.9856
val Loss: 0.0980 Acc: 0.9675
train Loss: 0.0256 Acc: 0.9919
val Loss: 0.1031 Acc: 0.9658
train Loss: 0.0142 Acc: 0.9966
val Loss: 0.1276 Acc: 0.9640
train Loss: 0.0148 Acc: 0.9969
val Loss: 0.1019 Acc: 0.9667
train Loss: 0.0139 Acc: 0.9947
val Loss: 0.1207 Acc: 0.9623
train Loss: 0.0184 Acc: 0.9940
val Loss: 0.1003 Acc: 0.9684
train Loss: 0.0115 Acc: 0.9969
val Loss: 0.1318 Acc: 0.9614
train Loss: 0.0066 Acc: 0.9984
val Loss: 0.1172 Acc: 0.9675
train Loss: 0.0130 Acc: 0.9969
val Loss: 0.1168 Acc: 0.9658
train Loss: 0.0066 Acc: 0.9987
val Loss: 0.1160 Acc: 0.9675
train Loss: 0.0070 Acc: 0.9981
val Loss: 0.1188 Acc: 0.9693
train Loss: 0.0094 Acc: 0.9978
val Loss:

In [79]:
torch.save(model.state_dict(), 'resnet_v2_fall_detection.pth')
